## SQP

In [1]:
# QP

# Linesearch with penalty

# Active set strategy

# BFGS for approximating Hessian



In [2]:
# Import libraries
import torch as t
import numpy as np
from matplotlib import pyplot as plt

import torch.nn as nn
from torch.autograd import Variable
from torch.autograd.functional import jacobian, hessian

In [13]:
# Part 1
"""
Decision variable: x0
State variables: x1
"""



m = 2 # No. of constraints
n = 2 # No. of variables
d = n - m # No. of decision variables


def f(x):
    # Define the objective function
    f = lambda x: x[0] ** 2 + (x[1] - 3) ** 2
    # Define the constraints
    g1 = lambda x: (x[1] ** 2) - (2 * x[0])
    g2 = lambda x: (x[1] - 1) ** 2 + (5 * x[0]) - 15
    
    return f(x), g1(x), g2(x)


def Lag(x, mu):
    # Define the Lagrangian
    # L = lambda x: x[0] ** 2 + (x[1] - 3) ** 2 + mu[0] * ((x[1] ** 2) - (2 * x[0])) + mu[1] * ((x[1] - 1) ** 2 + (5 * x[0]) - 15)
    
        # Define the objective function
    f = lambda x: x[0] ** 2 + (x[1] - 3) ** 2
    # Define the constraints
    g1 = lambda x: (x[1] ** 2) - (2 * x[0])
    g2 = lambda x: (x[1] - 1) ** 2 + (5 * x[0]) - 15

    return f(x) + mu.T @ t.tensor([[g1(x)], [g2(x)]])
    # return f(x) + mu[0] * g1(x) + mu[1] * g2(x)


In [14]:
# Compute Jacobian
def jac(x, n=n):
    J = t.zeros((m+1, n))
    for i in range(m+1):
       J[i] =  jacobian(f, (x))[i] # 'jacobian' function in Pytorch returns a tuple of tensors. Copying each tensor slice into a new tensor for the ease of indexing.
    return J


# Compute Jacobian
def jacL(x, mu, n=n):
    J = t.zeros((1, n))
    # for i in range(n):
    J[0] =  jacobian(Lag, (x, mu))[0] # 'jacobian' function in Pytorch returns a tuple of tensors. Copying each tensor slice into a new tensor for the ease of indexing.
    return J

# Evaluate Constraints
def hFunc(x, m=m, n=n):
    H = t.zeros((m, 1))
    for i in range(m):
        H[i] =  f(x)[d + i]
    return H

# Part 3
# Calculate the Reduced gradient from the jacobian
dfdd = lambda J: J[0,0] - J[0,1:] @ t.pinverse(J[1:,1:]) @ J[1:,0]

In [66]:
x = t.tensor([1., 1.], dtype=t.float64, requires_grad=True)
mu = t.tensor([0., 0.], dtype=t.float64, requires_grad=True)

W = t.eye(n)
# mu = t.tensor([0., 0.], dtype=t.float64)

# jacobian(Lag, (x, mu))


J =  jacL(x, mu)

# J =  jac(x)
# # Lag(x)
A = jac(x)[1:, :]
A
J

tensor([[ 2., -4.]])

In [145]:
A = t.tensor([[-100],[10],[-1]])

In [147]:
val1 , idx1 = t.max(hFunc(x) > 0, 0)
idx1

tensor([0])

In [139]:
ans.sum().item()

2

In [128]:
A[idx]

tensor([[1]])

In [80]:
mu = t.tensor([[-1], [0], [-5]])

In [81]:
mu<0

tensor([[ True],
        [False],
        [ True]])

In [82]:
val, idx = t.min(mu, 0)
idx

tensor([2])

In [83]:
mu[idx]

tensor([[-5]])

In [91]:
x = t.tensor([0., 0.], dtype=t.float, requires_grad=True)
mu = t.tensor([0., 0.], dtype=t.float, requires_grad=True)
m = t.eye(n)

_ , A = Lag(x, mu)

A

tensor([[  0.],
        [-14.]])

In [84]:
_, idx1 = t.max(A > 0, 0)
idx1


tensor([0])

In [89]:
mu = t.tensor([[0], [5]])
val2, idx2 = t.min(mu, 0)

In [155]:
active = []
add = []
remove = []

In [156]:
val1 , idx1 = t.max(hFunc(x) > 0, 0)

if val1 == True:
    add.append(idx1.item())

In [157]:
val2 , idx2 = t.max(mu < 0, 0)

if val2 == True:
    remove.append(idx2.item())

In [158]:
active.append(add)
active.remove(remove)

ValueError: list.remove(x): x not in list

In [62]:
def QP(x, W, A):
    # W = t.eye(n)
    A = jac(x)[1:, :]
    fx = jac(x)[0]
    h = hFunc(x)


    C = t.vstack((t.hstack((W, A.T)), t.hstack((A, t.zeros(A.shape[0], A.shape[0])))))
    C

    D = - t.vstack((fx.reshape(-1, 1), h)) # Check if this negative sign is important
    D

    X = t.linalg.solve(C, D)
    s = X[:n, :]
    mu = X[n:, :]
    
    return s, mu

In [ ]:
def activeSet(W, x, s, dx, mu):
    (t.max(A) == 0)
    

In [55]:
# Armijo Line 
def F(alpha, x, wj, A):
    F = f(x)[0] + t.sum(wj.T @ t.max(t.tensor([0]), A))
    return F

def phi(alpha, x, wj, A, t0=0.5):
    phi = F(alpha, x, wj, A) + t0 * alpha * dFda
    
    return phi

def dFda(alpha, x, s, wj, A):
    J =  jac(x)
    
    dgdx = J[1:, :]
    
    dgda = dgdx @ s
    
    dgda[(t.max(t.tensor([0]), A) == 0)] = 0
    
    dFda = J[0, :].T @ s + t.sum(wj.T @ t.max(t.tensor([0]), A))
    
    return dFda

def lineSearch(x, s, mu, K=25):
    alpha = 1
    i = 0
    wj = t.abs(mu)
    
    while F(alpha, x, wj, A) > phi(alpha, x, wj, A) and i < K:
        alpha = 0.5 * alpha
        wj = t.max(t.abs(mu), 0.5 * (wj + t.abs(mu)))
        i += 1
    return alpha

In [ ]:
def BFGS(W, x, s, dx, mu):
    Lx0 =  jacL(x - dx, mu)
    Lx1 =  jacL(x, mu)
    Q = dx.T @ W @ dx
    if dx.T @ (Lx1 - Lx0) >= 0.2 * Q:
        theta = 1
    else:
        theta = (0.8 * Q) / (Q - dx.T @ (Lx1 - Lx0))
    
    y = theta * (Lx1 - Lx0) + (1 - theta) * (W @ dx)
    
    W = W + ((y @ y.T) / (y.T @ s)) - ((W @ s @ s.T @ W) / (s.T @ H @ s))
    
    return W
    
    